In [3]:
import numpy as np
from keras import Sequential
from keras.layers import Dense, Dropout

Pretraining the trans function for a garch(2,1) model

In [16]:
# create dataset
s1 = 50
s2 = 50
s3 = 23
const = np.random.normal(loc=0, scale=0.001, size=s1 * s2 * s3)
q1s = np.tile(np.linspace(0.03, 0.20, s1), s2 * s3) + np.random.normal(loc=0, scale=0.002, size=s1 * s2 * s3)
q2s = np.tile(np.linspace(0.64, 0.96, s2), s1 * s3) + np.random.normal(loc=0, scale=0.002, size=s1 * s2 * s3)
p1s = np.tile(np.linspace(0.5, 1, s3), s1 * s2) + np.random.normal(loc=0, scale=0.002, size=s1 * s2 * s3)
r1s = np.random.normal(loc=0, scale=0.03, size=s1 * s2 * s3)
r2s = np.random.normal(loc=0, scale=0.03, size=s1 * s2 * s3)
cv1s = np.random.normal(loc=0, scale=0.03, size=s1 * s2 * s3)
xs = np.zeros((s1 * s2 * s3, 7))
ys = np.zeros((s1 * s2 * s3, 1))
i = 0
for q1_i in range(0, s1):
    for q2_i in range(0, s2):
        for p1_i in range(0, s3):
            xs[i][0] = const[i]
            xs[i][1] = q1s[i]
            xs[i][2] = q2s[i]
            xs[i][3] = p1s[i]
            xs[i][4] = cv1s[i]
            xs[i][5] = r2s[i]
            xs[i][6] = r1s[i]
            ys[i][0] = const[i] + q1s[i] * r1s[i] ** 2 + q2s[i] * r2s[i] ** 2 + p1s[i] * cv1s[i] ** 2
            i += 1

In [17]:
model = Sequential()
# 4 parameters, 1 prev vols, 2 prev innovations = 7 inputs
model.add(Dense(60, input_shape=(7,), activation='sigmoid'))
model.add(Dropout(.2, input_shape=(60,)))
model.add(Dense(30, input_shape=(60,), activation='sigmoid'))
model.add(Dropout(.2, input_shape=(30,)))
model.add(Dense(1, input_shape=(30,), activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

model.fit(xs, ys, epochs=8, batch_size=25)

Epoch 1/8
2300/2300 [==============================] - 3s 1ms/step - loss: 0.0194 - accuracy: 0.0000e+00
Epoch 2/8
2300/2300 [==============================] - 3s 1ms/step - loss: 2.5906e-04 - accuracy: 0.0000e+00
Epoch 3/8
2300/2300 [==============================] - 3s 1ms/step - loss: 5.1354e-05 - accuracy: 0.0000e+00
Epoch 4/8
2300/2300 [==============================] - 3s 1ms/step - loss: 1.4699e-05 - accuracy: 0.0000e+00
Epoch 5/8
2300/2300 [==============================] - 3s 1ms/step - loss: 6.4511e-06 - accuracy: 0.0000e+00
Epoch 6/8
2300/2300 [==============================] - 3s 1ms/step - loss: 4.1478e-06 - accuracy: 0.0000e+00
Epoch 7/8
2300/2300 [==============================] - 3s 1ms/step - loss: 3.4945e-06 - accuracy: 0.0000e+00
Epoch 8/8
2300/2300 [==============================] - 3s 1ms/step - loss: 3.2850e-06 - accuracy: 0.0000e+00


In [18]:
dropout_layers = 0
for layer in range(0, 3):
    np.save(f"./pretrained_weights/garch_trans_layer_{layer}_weights", model.layers[layer + dropout_layers].get_weights()[0])
    np.save(f"./pretrained_weights/garch_trans_layer_{layer}_biases", model.layers[layer + dropout_layers].get_weights()[1])
    dropout_layers += 1

In [15]:
import torch
import numpy as np

t1 = torch.Tensor([[[1,2],[3,4]]])
t2 = torch.Tensor([[[10],[11]]])
torch.concat((t1, t2), dim=2)

t2.tile((10,1,1))

arr1 = np.array([[[1],[2],[3]], [[1],[2],[4]]])

max_values = arr1.max(axis=1).reshape((len(arr1), ))

threshold = np.percentile(max_values, 95)

indices = np.where(max_values > threshold)[0]
indices

array([1], dtype=int64)